In [13]:
import pandas as pd
import os
import json
import numpy as np
import re

In [14]:
## reading the Survey File
#surveyData = pd.read_csv("C:\Projects\MSS\Loading _AmenityData_from_Survey\Test.csv",sep="," , skiprows=1)
surveyData = pd.read_csv("C:\project\snaplogic\MSS\Amenities\AmenityClubSurvey FINAL v2 Season Cleaned_deletedFirstTwoColumn.csv",sep="," , skiprows=1)
surveyData.head()

## Reading the Config File and Extracting the Information from there
config_file = open("C:\project\snaplogic\MSS\Amenities\ClubCorp\SteamRoom\SteamRoomColMapping.json")
SteamRoom_config = json.load(config_file)

columnName = ""

In [15]:
def addToDayTimeMap (phonebook, key, value):
    if key in phonebook:
        x =  phonebook[key]
        phonebook[key] = x + "," + value
        
    else :
        phonebook[key] = value

In [16]:
def prepareHoursOfOperationJson(hoursOfOperationInfo, surveyData, isFirstDiningRoom):
    
    allHoursOfOperation = []
    
    for i in range(0,len(surveyData)) : 
        hoursOfOperationForEachRow = []
        hoursOfOperation = {}
        hoursOfOperationValue = {}

        season = "All Season"
        ## adding season value
        if season and not pd.isnull(season):
            seasonValue = {}
            seasonValue["value"] = season
            hoursOfOperationValue["Season"] = [seasonValue]
        
        
        ## creating day time map so that days with same open and closed time can handled properly
        dayTimeMap = {}    
        for j in range(0, len(hoursOfOperationInfo)):
            
            dayInfo = hoursOfOperationInfo[j]
            day = dayInfo["Day"]
            openTimeIndex = dayInfo["OpenTime"]
            closeTimeIndex = dayInfo["CloseTime"]
            
            if not isFirstDiningRoom :
                openTimeIndex = openTimeIndex+  columnIndexIncreaseCountFor2ndDining;
                closeTimeIndex = closeTimeIndex+  columnIndexIncreaseCountFor2ndDining;
                
            startTimeRaw = surveyData.iat[i, openTimeIndex]
            endTimeRaw = surveyData.iat[i, closeTimeIndex]
            
            startTime = ""
            endTime = ""
            if pd.notnull(startTimeRaw):
                startTime = str(startTimeRaw).strip()
            
            if pd.notnull(endTimeRaw):
                endTime = str(endTimeRaw).strip()
                
            
            
            if startTime == "Closed" or endTime == "Closed" :
                continue
            else :  
                addToDayTimeMap(dayTimeMap, startTime + "#$" + endTime, day)
            
        
        dayAndTimingArray = []
        
        for startAndEndTime, days in dayTimeMap.items():
            
            startTime,endTime = startAndEndTime.split("#$")
            openDays = days.split(",")
            ## Adding Day and Timing
            dayAndTiming = {}
            dayAndTimingValue = {}
            for day in openDays :
                dayValue = {}
                dayValue["value"] = True
                dayAndTimingValue[day] = [dayValue]

            timing = {}
            timingValue = {}

            if startTime and not pd.isnull(startTime): 
                startTimeDict ={}
                startTimeDict["value"] = startTime
                timingValue["StartTime"] =  [startTimeDict]

            if endTime and not pd.isnull(endTime):
                endTimeDict ={}
                endTimeDict["value"] = endTime
                timingValue["EndTime"] =  [endTimeDict]
            
            ## Adding day part as "All Day"
            dayPart ={}
            dayPart["value"] = "All Day"
            timingValue["DayPart"] =  [dayPart]


            if timingValue :
                timing["value"] = timingValue
                dayAndTimingValue["Timing"] = [timing]

            if dayAndTimingValue : 
                dayAndTiming["value"] = dayAndTimingValue    
                dayAndTimingArray.append(dayAndTiming)
        
        hoursOfOperationValue["DayAndTiming"] = dayAndTimingArray
        if hoursOfOperationValue :
            hoursOfOperation["value"] = hoursOfOperationValue
            
        hoursOfOperationForEachRow.append(hoursOfOperation)
        allHoursOfOperation.append(json.dumps(hoursOfOperationForEachRow))
    
    return allHoursOfOperation

In [17]:
def determineBooleanValues (dataFrame):
    val = False
    if pd.notnull(dataFrame[columnName]) :
        if dataFrame[columnName] == columnName:
            val = True
    return val

In [18]:
## Getting the output File name
output_fileName = SteamRoom_config['outputFileName']


all_column_names = []
SteamRoom_related_column_names = []

columnIndexForSteamRoom = []

common_column_count = len(SteamRoom_config['CommonColumn'])

for i in range(0, common_column_count):
    columnIndexForSteamRoom.append(SteamRoom_config['CommonColumn'][i]['columnIndex'])
    all_column_names.append(SteamRoom_config['CommonColumn'][i]['outputColumnName'])

SteamRoom_column_count = len(SteamRoom_config['SteamRoomRelatedColumn'])

for j in range(0, SteamRoom_column_count):
    columnIndexForSteamRoom.append(SteamRoom_config['SteamRoomRelatedColumn'][j]['columnIndex'])
    all_column_names.append(SteamRoom_config['SteamRoomRelatedColumn'][j]['outputColumnName'])
    SteamRoom_related_column_names.append(SteamRoom_config['SteamRoomRelatedColumn'][j]['outputColumnName'])
   
## getting Hours of Related column information from config file
hoursOfOperationInfo = SteamRoom_config['HoursOfOperationInformation']


## Data for Steam Room
SteamRoom = surveyData.iloc[:, columnIndexForSteamRoom]
SteamRoom.columns = all_column_names

SteamRoom['Index']=1

hrsOfOperation = prepareHoursOfOperationJson(hoursOfOperationInfo, surveyData, True)
SteamRoom['HrsOfOpeartion'] = hrsOfOperation

SteamRoom['Status'] = 'Active'
SteamRoom['Type'] = 'Health & Wellness'
SteamRoom['Name'] = 'Steam Room'

for x  in SteamRoom_related_column_names:
    columnName = x
    SteamRoom[columnName + '_Boolean'] = SteamRoom.apply(determineBooleanValues, axis=1)


SteamRoom.to_csv("C:/project/snaplogic/MSS/Amenities/" + output_fileName)

C:\Users\eshita\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\eshita\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\eshita\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas